# Heat Equation

Solve the differential equation
\begin{align}
    \partial_t u(t,x) - \partial_x^2 u(t,x) &= 0, && t \in (0,T), \, x \in (x_\mathrm{min}, x_\mathrm{max}),
    \\
    u(0,x) &= u_0(x), && x \in [x_\mathrm{min}, x_\mathrm{max}]
\end{align}
with appropriate boundary conditions.

## Constant Dirichlet Boundary Conditions

Solve the heat equation with constant Dirichlet boundary conditions 
\begin{align}
    u(t,x_\mathrm{min}) &= u_0(x_\mathrm{min}), && t \in (0,T),
    \\
    u(t,x_\mathrm{max}) &= u_0(x_\mathrm{max}), && t \in (0,T).
\end{align}

### Using `SummationByPartsOperators.jl`

In [ ]:
using SummationByPartsOperators, OrdinaryDiffEq
using Plots, LaTeXStrings; pyplot()

xmin, xmax = -π, π
N = 512
acc_order = 4
tspan = (0., 10.)
# source of coefficients
source = MattssonSvärdShoeybi2008()
ode_alg = Tsit5()

u₀(x) = -(x - 0.5)^2 + 1/12

D = derivative_operator(source, 2, acc_order, xmin, xmax, N)
x = D.grid
u0 = u₀.(x)

function rhs!(t, u, du)
    A_mul_B!(du, D, u)
    @inbounds du[1] -= (u[1] - u₀(xmin)) / D.coefficients.left_weights[1]
    @inbounds du[end] -= (u[end] - u₀(xmax)) / D.coefficients.right_weights[1]
end

ode = ODEProblem(rhs!, u0, tspan)
sol = solve(ode, ode_alg, save_everystep=false, tstops=0:0.01:10)
@time sol = solve(ode, ode_alg, save_everystep=false, tstops=0:0.01:10)

# try to plot the solution at different time points using
plot(x, [sol(i) for i in 0:1:10])

### Using `DiffEqOperators.jl`

In [ ]:
using DiffEqOperators, OrdinaryDiffEq
using Plots, LaTeXStrings; pyplot()

xmin, xmax = -π, π
N = 512
acc_order = 4
tspan = (0., 10.)
ode_alg = Tsit5()

u₀(x) = -(x - 0.5)^2 + 1/12

x = linspace(xmin, xmax, N)
u0 = u₀.(x)
L = DerivativeOperator{Float64}(2, acc_order, 2π/511, 512, :Dirichlet, :Dirichlet; BC=(u0[1],u0[end]))

ode = ODEProblem(L, u0, tspan)
sol = solve(ode, ode_alg, save_everystep=false, tstops=0:0.01:10)
@time sol = solve(ode, ode_alg, save_everystep=false, tstops=0:0.01:10)

# try to plot the solution at different time points using
plot(x, [sol(i) for i in 0:1:10])